# TP : Proximal coordinate descent method on regression models

#### Authors: S. Gaiffas, A. Gramfort

## Aim

The aim of this material is to code 
- proximal coordinate descent

for 
- Lasso / L1 linear regression
- non-negative least squares (NNLS)

models.

The proximal operators we will use are the 
- L1 penalization
- indicator function of $\mathbb{R}_+$

## VERY IMPORTANT

- This work **must be done by pairs of students**.
- **Each** student must send their work **before the 23th of october at 23:59**, using the **moodle platform**.
- This means that **each student in the pair sends the same file**
- On the moodle, in the "Optimization for Data Science" course, you have a "devoir" section called **Rendu TP du 17 octobre 2016**. This is where you submit your jupyter notebook file. 
- The **name of the file must be** constructed as in the next cell

# Gentle reminder: no evaluation if you don't respect this EXACTLY

### How to construct the name of your file

In [ ]:
# Change here using YOUR first and last names
fn1 = "camille"
ln1 = "masset"
fn2 = "boris"
ln2 = "muzellec"

filename = "_".join(map(lambda s: s.strip().lower(), 
                        ["tp_cd", ln1, fn1, "and", ln2, fn2])) + ".ipynb"
print(filename)

In [ ]:
## to embed figures in the notebook
%matplotlib inline

## Part 0 : Introduction

We'll start by generating sparse positive vectors and simulating data

### Getting sparse coefficients

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=2)  # to have simpler print outputs with numpy

In [ ]:
n_features = 50
n_samples = 1000
idx = np.arange(n_features)
coefs = (idx % 2) * np.exp(-idx / 10.)
coefs[20:] = 0.
plt.stem(coefs)
plt.title("Parameters / Coefficients")

### Functions for the simulation of the models

In [ ]:
from numpy.random import multivariate_normal
from scipy.linalg.special_matrices import toeplitz
from numpy.random import randn


def simu_linreg(coefs, n_samples=1000, corr=0.5):
    """Simulation of a linear regression model
    
    Parameters
    ----------
    coefs : `numpy.array`, shape=(n_features,)
        Coefficients of the model
    
    n_samples : `int`, default=1000
        Number of samples to simulate
    
    corr : `float`, default=0.5
        Correlation of the features

    Returns
    -------
    A : `numpy.ndarray`, shape=(n_samples, n_features)
        Simulated features matrix. It samples of a centered Gaussian 
        vector with covariance given by the Toeplitz matrix
    
    b : `numpy.array`, shape=(n_samples,)
        Simulated labels
    """
    # Construction of a covariance matrix
    cov = toeplitz(corr ** np.arange(0, n_features))
    # Simulation of features
    A = multivariate_normal(np.zeros(n_features), cov, size=n_samples)
    # Simulation of the labels
    b = A.dot(coefs) + randn(n_samples)
    return A, b

## Proximal operators and Solver


We remind that the proximal operator of a fonction $g$ is given by:

$$
\text{prox}_g(y, t) = \arg\min_x \Big\{ \frac 12 \|x - y\|_2^2 + t g(x) \Big\}.
$$

where $t \geq 0$ is a non-negative number.
We have in mind to use the following cases

- Lasso penalization, where $g(x) = s \|x\|_1$
- Indicator function of $\mathbb{R}_+$, where $g(x) = i_{x \geq 0}(\cdot)$

where $s \geq 0$ is a regularization parameter.

We want to minimize:
$$
\arg\min_x F(x)
$$
with
$$
 F(x) = \frac{1}{2} \|b - Ax\|^2 + g(x)
$$

## Questions

- Code a function that computes $g(x)$ and $\text{prox}_g(x)$ for in both cases
- Justify why proximal coordinate descent can be applied to obtain a minimum of such objective functions.
- Starting from the code provided in the notebook presented during the coordinate descent course as well as the code below, implement a proximal coordinate method for both penalties.
- Evaluate qualitatively the convergence when varying the conditioning of the problem.
- Bonus: Try to show that coordinate is much less affected by bad conditioning that proximal gradient descent.

### You are expected to implement the smart residuals updates !

### You are very welcome to reuse everything you did for TP1 !

#### Lasso penalisation and proximal operator

In [ ]:
def prox_lasso(x, s, t=1.):
    """Proximal operator for the Lasso at x with strength t"""
    return np.multiply(np.sign(x), np.maximum(np.absolute(x) - s*t, 0))
    
def lasso(x, s):
    """Value of the Lasso penalization at x with strength t"""
    return s*np.linalg.norm(x, 1)

#### Indicator function penalisation and proximal operator

In [ ]:
def indicator(x, s):
    """Value of the indicator function of R+ penalisation at x with strength s"""
    if s == 0:
        return 0
    return np.inf if any(x < 0) else 0

def prox_indicator(x, s, t=1.):
    """Proximal operator for indicator function of R+ at x with strength s"""
    return x if t*s == 0 else np.maximum(x, 0)

We can use the proximal coordinate descent, because all the conditions that we saw in class are gathered:
* $f$ is convex and has a Lipschitz-continuous gradient;
* $g$ is convex and separable (non-smooth in our case).

## Proximal coordinate descent

In [ ]:
def cd_linreg(x0, A, b, g, prox_g, s=0., n_iter=50,
              x_true=coefs, verbose=True):
    """Proximal gradient descent algorithm

    Minimize :
    
    1/2 ||b−Ax||^2 + s * g(x)
    
    with coodinate descent.
    """
    x = x0.copy()
    x_new = x0.copy()
    r = b - A.dot(x)
    n_samples, n_features = A.shape
    
    # estimation error history
    errors = []
    # objective history
    objectives = []
    # Current estimation error
    err = np.linalg.norm(x - x_true) / np.linalg.norm(x_true)
    errors.append(err)
    # Current objective
    obj = 0.5 * np.linalg.norm(b - A.dot(x))**2 + g(x, s)
    objectives.append(obj)

    if verbose:
        print("Lauching Coordinate Descent solver...")
        print(' | '.join([name.center(8) for name in ["it", "obj", "err"]]))
    
    L = np.array([np.dot(A[:, i], A[:, i]) for i in range(A.shape[1])]) # we pre-compute Ai.T * Ai = Lipschitz constants
    
    for k in range(n_iter + 1):
        i = k % n_features # cyclic feature selection
        
        x_new[i] += A[:, i].T.dot(r) / L[i]
        x_new[i] = prox_g(x_new[i], s, 1/L[i])
        r -= (x_new[i] - x[i]) * A[:, i]
        x[i] = x_new[i]
        
        obj = 0.5 * np.linalg.norm(b - A.dot(x))**2 + g(x, s)
        err = np.linalg.norm(x - x_true) / np.linalg.norm(x_true)
        errors.append(err)
        objectives.append(obj)
        if k % 10 == 0 and verbose:
            print(' | '.join([("%d" % k).rjust(8), 
                              ("%.2e" % obj).rjust(8), 
                              ("%.2e" % err).rjust(8)]))
    return x, objectives, errors

## Proximal gradient descent (for comparison purposes)

In [ ]:
def ista(x0, A, b, g, prox_g,  s=0., n_iter=50,
         x_true=coefs, verbose=True):
    """Proximal gradient descent algorithm
    """
    x = x0.copy()
    x_new = x0.copy()
    n_samples, n_features = A.shape
    
    # estimation error history
    errors = []
    # objective history
    objectives = []
    # Current estimation error
    err = np.linalg.norm(x - x_true) / np.linalg.norm(x_true)
    errors.append(err)
    # Current objective
    obj = 0.5 * np.linalg.norm(b - A.dot(x))**2 + g(x, s)
    objectives.append(obj)
    
    #Lipschitz constant
    L = np.linalg.norm(A.T.dot(A), 2)
    
    if verbose:
        print ("Lauching ISTA solver...")
        print (' | '.join([name.center(8) for name in ["it", "obj", "err"]]))
    for k in range(n_iter + 1):

        x = prox_g(x - 1/L*A.T.dot(A.dot(x)-b), s, 1/L)
        
        obj = 0.5 * np.linalg.norm(b - A.dot(x))**2 + g(x, s)
        err = np.linalg.norm(x - x_true) / np.linalg.norm(x_true)
        errors.append(err)
        objectives.append(obj)
        if k % 10 == 0 and verbose:
            print (' | '.join([("%d" % k).rjust(8), 
                              ("%.2e" % obj).rjust(8), 
                              ("%.2e" % err).rjust(8)]))
    return x, objectives, errors

# A (reasonably) well conditionned problem

We fix number of iterations, and multiply it by the number of features in the case of coordinate descent for fair comparision.

In [ ]:
n_iter = 10

In [ ]:
n_features = 50
n_samples = 1000
idx = np.arange(n_features)
coefs = (idx % 2) * np.exp(-idx / 10.)
coefs[20:] = 0.
plt.stem(coefs)
plt.title("Parameters / Coefficients")

In [ ]:
A, b = simu_linreg(coefs, n_samples=n_samples)

## Proximal gradient descent

In [ ]:
x_hat, obj, err = ista(np.zeros(n_features), A, b, lasso, prox_lasso, s = 1e-02, n_iter=n_iter)

In [ ]:
plt.plot(np.arange(len(err)), err, label="Error")

In [ ]:
plt.plot(np.arange(len(obj)), obj, label="Objective")

## Proximal coordinate descent

In [ ]:
x_hat, obj, err = cd_linreg(np.zeros(n_features), A, b, lasso, prox_lasso, s = 1e-02, n_iter=n_iter*n_features)

In [ ]:
plt.plot(np.arange(len(err)), err, label="Error")

In [ ]:
plt.plot(np.arange(len(obj)), obj, label="Objective")

# A badly conditionned problem

In [ ]:
n_features = 50
n_samples = 1000
idx = np.arange(n_features)
coefs = (idx % 2) * np.exp(-idx / 10.)
coefs[20:] = 0.
coefs[3]*=10
coefs[9]*=10
plt.stem(coefs)
plt.title("Parameters / Coefficients")

In [ ]:
A, b = simu_linreg(coefs, n_samples=n_samples)

## Proximal gradient descent

In [ ]:
x_hat, obj, err = ista(np.zeros(n_features), A, b, lasso, prox_lasso, s = 1e-02, n_iter=n_iter)

In [ ]:
plt.plot(np.arange(len(obj)), obj, label="Objective")

## Proximal coordinate descent

In [ ]:
x_hat, obj, err = cd_linreg(np.zeros(n_features), A, b, lasso, prox_lasso, s = 1e-02, n_iter=n_iter*n_features)

In [ ]:
plt.plot(np.arange(len(obj)), obj, label="Objective")

# Conclusion

As we can see, in the case of a well-conditionned problem, both methods achieve roughly the same objective value in the same running time, which was predictable since in a well-conditionned problem coordinate descent does not fully take advantage of using coordinate-tailored Lipschitz constant.

However, in our badly-conditionned problem, coordinate descent achieves an objective value which is half that of proximal gradient descent. This shows that coordinate descent is more suited that proximal gradient descent in the case of poor conditionning, as it uses a different Lipschitz constant for each feature.

Further, in the case of coordinate descent we observe that nothing hapens when we iterate over those features that have no importance. In our second example, were only 2 features are significant, we see that an improvement is achieved only twice every 50 iterations, i.e. when we descend on the 2 significant features.